Calculo com machine learning utilizando cluster

In [3]:
import pandas as pd
from sklearn.cluster import KMeans
import numpy as np
import re

def AlterarGastos(dataframe):
    dataframe['Gasto'] = dataframe['Gasto'].apply(lambda x: float(re.sub(r'KWh$', '', x).replace(',', '.')) if isinstance(x, str) else x)
    return dataframe

# Lendo arquivos CSV
df1 = AlterarGastos(pd.read_csv('../Files/PresetArquivoTrain.csv', delimiter=';'))
df3 = AlterarGastos(pd.read_csv('../Files/PresetArquivoTeste.csv', delimiter=';'))

# Agrupando dados por eletrodoméstico e computando a média de gastos
df1_grouped = df1.groupby(['Eletrodomestico']).mean()

# Ajustando o modelo KMeans com os valores médios de gastos
kmeans = KMeans(n_clusters=len(df1_grouped)).fit(df1_grouped[['Gasto']])

# Prevendo os clusters dos valores médios dos gastos
df1_grouped['cluster'] = kmeans.predict(df1_grouped[['Gasto']])

# Prevendo o cluster de cada valor de gasto do segundo arquivo
df3['cluster'] = kmeans.predict(df3[['Gasto']])

# Computando as médias de gastos por cluster do segundo arquivo
df3_clustered = df3.groupby(['Eletrodomestico', 'cluster']).mean().reset_index()

# Merge entre os dados do primeiro arquivo e os dados do segundo arquivo com as médias de gastos por cluster
df_merged = pd.merge(df1_grouped.reset_index(), df3_clustered, on=['Eletrodomestico', 'cluster'], suffixes=('_train', '_test'))

# Calculando o desvio em relação à média de gastos por cluster para cada linha do segundo arquivo
df3['DesvioPadrao'] = np.abs(df3['Gasto'] - df_merged['Gasto_test'])

# Selecionando as linhas do segundo arquivo que estão fora do padrão
limite = 0.1  # limite de 10% de desvio em relação à média por cluster
df_fora_do_padrao = df3[df3['DesvioPadrao'] > limite]

# Exibindo as linhas do segundo arquivo que estão fora do padrão
print(df_fora_do_padrao)

#Salvar modelo para utilizar sem ter que gerar novamente
from joblib import dump

dump(kmeans, '../Model/clusterbrain.joblib')

c:\Users\Alan-\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


     Eletrodomestico      Comodo  Gasto        Data  cluster  DesvioPadrao
0          Geladeira     Cozinha  500.0  13/03/2023        2         415.0
1    Ar condicionado      Quarto   90.0  13/03/2023        2          83.0
2   Maquina de Lavar  Lavanderia   45.0  13/03/2023        4          10.0
3          Televisao        Sala   35.0  13/03/2023        1          20.0
4  Chuveiro eletrico    Banheiro   30.0  13/03/2023        9          22.0
5         Ventilador      Quarto   20.0  13/03/2023        5           5.0
6         Microondas     Cozinha   25.0  13/03/2023        3          13.0


C:\Users\Alan-\AppData\Local\Temp\ipykernel_25788\3314808566.py:18: ConvergenceWarning: Number of distinct clusters (11) found smaller than n_clusters (14). Possibly due to duplicate points in X.
  kmeans = KMeans(n_clusters=len(df1_grouped)).fit(df1_grouped[['Gasto']])


FileNotFoundError: [Errno 2] No such file or directory: './Model/clusterbrain.joblib'

Modelo ja treinado, so rodar os predicts

Exemplos no campo de cima, onde ta o treinamento

In [9]:
from joblib import load

model = load('../Model/clusterbrain.joblib')

Calculo sem machine learning

In [2]:
import pandas as pd
import re

def AlterarGastos(dataframe):
    dataframe['Gasto'] = dataframe['Gasto'].apply(lambda x: float(re.sub(r'KWh$', '', x).replace(',', '.')) if isinstance(x, str) else x)
    return dataframe

# Lendo arquivos CSV
df1 = AlterarGastos(pd.read_csv('../Files/PresetArquivoTrain.csv', delimiter=';'))
df3 = AlterarGastos(pd.read_csv('../Files/PresetArquivoTeste.csv', delimiter=';'))

# Cria um dicionário com as médias de gasto por eletrodoméstico presentes no DataFrame df1
media_gastos = {}
for eletrodomestico in df1['Eletrodomestico'].unique():
    media_gastos[eletrodomestico] = df1[df1['Eletrodomestico'] == eletrodomestico]['Gasto'].mean()

# Cria um dicionário com as médias de gasto por eletrodoméstico presentes no DataFrame df3
media_gastos_novos = {}
for eletrodomestico in df3['Eletrodomestico'].unique():
    media_gastos_novos[eletrodomestico] = df3[df3['Eletrodomestico'] == eletrodomestico]['Gasto'].mean()

# Adiciona a média de gasto correspondente a cada linha do DataFrame df3
df3['MediaGasto'] = df3['Eletrodomestico'].apply(lambda x: media_gastos_novos[x] if x in media_gastos_novos else media_gastos[x])

# Calcula o desvio de cada linha em relação à média correspondente
df3['DesvioPadrao'] = abs(df3['Gasto'] - df3['MediaGasto'])

# Separa as linhas com desvio de mais de 10% da média em um DataFrame separado
limite = 0.1  # limite de 10% de desvio em relação à média
df_fora_do_padrao = df3[df3['DesvioPadrao'] > limite]

# Exibe as linhas fora do padrão
print(df_fora_do_padrao)

      Eletrodomestico    Comodo  Gasto        Data  MediaGasto  DesvioPadrao
0           Geladeira   Cozinha  500.0  13/03/2023  277.500000    222.500000
1     Ar condicionado    Quarto   90.0  13/03/2023   85.000000      5.000000
3           Televisao      Sala   35.0  13/03/2023   31.666667      3.333333
4   Chuveiro eletrico  Banheiro   30.0  13/03/2023   32.500000      2.500000
5          Ventilador    Quarto   20.0  13/03/2023   17.333333      2.666667
9      Forno Eletrico   Cozinha   50.0  13/03/2023   47.500000      2.500000
12    Ar condicionado      Sala   75.0  14/03/2023   85.000000     10.000000
13  Chuveiro eletrico  Banheiro   35.0  14/03/2023   32.500000      2.500000
14          Televisao    Quarto   25.0  14/03/2023   31.666667      6.666667
15         Ventilador      Sala   12.0  14/03/2023   17.333333      5.333333
17     Forno Eletrico   Cozinha   45.0  14/03/2023   47.500000      2.500000
21          Geladeira   Cozinha   55.0  15/03/2023  277.500000    222.500000